In [3]:
###calculate the dN/dS ratio of the genomes
###convert protein alignment into codon aligenment using pal2nal
###need perl package
from Bio import SeqIO
import re
import glob
import os 

In [ ]:
#build CDS dataset of the genomes set
#,target_file)
def get_full_gbk(proka_result,target_file):
    pattern = os.path.join(proka_result, '**', '*.ffn')
    fnn_files = glob.glob(pattern, recursive=True)
    with open(target_file, 'w') as outfile:
        for file in fnn_files:
            with open(file, 'r') as infile:
                records = SeqIO.parse(infile, 'fasta')
                SeqIO.write(records, outfile, 'fasta')
    print(f"整合完成，结果保存在 '{target_file}'")
get_full_gbk("/mnt/maple/ymchen/gene_loss/preflow/prokka_result","/mnt/maple/ymchen/gene_loss/preflow/cds_dataset.fna")

整合完成，结果保存在 '/mnt/maple/ymchen/gene_loss/preflow/cds_dataset.fna'


In [22]:
#sequence_file,target_file
def get_corresponding_cds(protein_file,dataset,target_file):
    id_list=[]
    container={}
    with open(protein_file) as p:
        for line in p:
            line=line.strip()
            if bool(re.search(r'>', line)):
                id = line[1:]
                id_list.append(id)
        #print(id_list)
    with open(dataset, 'r') as infile, open(target_file, 'w') as outfile:
        for record in SeqIO.parse(infile, 'fasta'):
            if record.id in id_list:
                container[record.id]=record
        ordered_container = {key: container[key] for key in id_list if key in container}
        for seq_id,sequence in ordered_container.items():
            SeqIO.write(sequence, outfile, 'fasta')
    #print("提取完成，结果保存在 'extracted_nucleotides.fasta'")
#get_corresponding_cds("/mnt/maple/ymchen/gene_loss/preflow/ortho/Results_Sep11/Single_Copy_Orthologue_Sequences/OG0001442.fa","/mnt/maple/ymchen/gene_loss/preflow/cds_dataset.fna","/mnt/maple/ymchen/gene_loss/preflow/example.fna")

In [24]:
### get full folder of correponding files 
def get_folder_correspond(ortholog_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(target_folder):
            pass
        else:
            raise
    for filename in os.listdir(ortholog_folder):
        true_name=filename.rsplit(".",1)[0]
        file_path = os.path.join(ortholog_folder, filename)
        get_corresponding_cds(file_path,"/mnt/maple/ymchen/gene_loss/preflow/cds_dataset.fna",f"{target_folder}/{true_name}.fna")
        
get_folder_correspond("/mnt/maple/ymchen/gene_loss/preflow/ortho/Results_Sep11/Single_Copy_Orthologue_Sequences/","/mnt/maple/ymchen/gene_loss/preflow/correspond_files")  

In [14]:
###now start using Pal2Nal to convert aa alignment into codon-based nucleotide alignment
def aa2codon(aa_align,fna_file,target_file):
    os.system(f"perl /mnt/maple/ymchen/pal2nal/pal2nal.pl {aa_align} {fna_file} -output fasta > {target_file} -codontable 11")
    print(f"perl /mnt/maple/ymchen/pal2nal/pal2nal.pl {aa_align} {fna_file} -output fasta > {target_file} -codontable 11")
def convert_aacodon_all(aa_folder,fna_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(target_folder):
            pass
        else:
            raise
    for filename in os.listdir(aa_folder):
        aa_name=filename.rsplit(".",1)[0]
        aa_path = os.path.join(aa_folder, aa_name+".fa")
        for dna_file in os.listdir(fna_folder):
            dna_name=dna_file.rsplit(".",1)[0]
            dna_path = os.path.join(fna_folder, dna_name+".fna")
            if dna_name==aa_name:
                aa2codon(aa_path,dna_path,f"{target_folder}/{dna_name}.fa")
#aa2codon("/mnt/maple/ymchen/gene_loss/preflow/msa/OG0001442.fa","/mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001442.fna","/mnt/maple/ymchen/gene_loss/preflow/OG0001442.fa")                
convert_aacodon_all("/mnt/maple/ymchen/gene_loss/preflow/msa","/mnt/maple/ymchen/gene_loss/preflow/correspond_files","/mnt/maple/ymchen/gene_loss/preflow/codon_align")        

#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001303.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001303.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#

#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001234.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001234.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#



perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001303.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001303.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001303.fa -codontable 11
perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001234.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001234.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001234.fa -codontable 11


#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001351.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001351.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#

#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001296.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001296.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#



perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001351.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001351.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001351.fa -codontable 11
perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001296.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001296.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001296.fa -codontable 11


#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001338.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001338.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#

#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0000910.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0000910.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#



...

perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0000926.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0000926.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0000926.fa -codontable 11
perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001281.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001281.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001281.fa -codontable 11
perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001036.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001036.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001036.fa -codontable 11


#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001036.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001036.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#

#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001293.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001293.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#



perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001293.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001293.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001293.fa -codontable 11
perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001350.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001350.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001350.fa -codontable 11


#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001350.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001350.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#

#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001418.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001418.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#

#------------------------------------------------------------------------#
#  Input files:  /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001298.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001298.fna
#  Codontable 11 is used
#------------------------------------------------------------------------#



perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001418.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001418.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001418.fa -codontable 11
perl /mnt/maple/ymchen/pal2nal/pal2nal.pl /mnt/maple/ymchen/gene_loss/preflow/msa/OG0001298.fa /mnt/maple/ymchen/gene_loss/preflow/correspond_files/OG0001298.fna -output fasta > /mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001298.fa -codontable 11


In [15]:
###reivse through trimal
def revise_msa_result(align_folder,revise_align_folder):
    for filename in os.listdir(align_folder):
        filepath = os.path.join(align_folder, filename)
        if os.path.isfile(filepath):
            os.system(f"trimal -in {filepath} -out {revise_align_folder}/{filename} -automated1")
revise_msa_result("/mnt/maple/ymchen/gene_loss/preflow/codon_align","/mnt/maple/ymchen/gene_loss/preflow/codon_align")

In [6]:
###convert format of alignment
def fas2phy(fas_file,phy_file):
    with open(phy_file, "w") as phylip_file:
        sequences = SeqIO.parse(fas_file, "fasta")
        SeqIO.write(sequences, phy_file, "phylip")
#fas2phy("/mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0000803.fa","/mnt/maple/ymchen/gene_loss/preflow/OG0000803.phy")
def convert_fas2phy_all(codon_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(target_folder):
            pass
        else:
            raise
    for filename in os.listdir(codon_folder):
        file_name=filename.rsplit(".",1)[0]
        file_path = os.path.join(codon_folder, filename)
        fas2phy(file_path,f"{target_folder}/{file_name}.phy")
convert_fas2phy_all("/mnt/maple/ymchen/gene_loss/preflow/codon_align","/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy")

In [6]:
###convert (new)
def fasta_to_phylip(fasta_file, phylip_file):
    sequences = list(SeqIO.parse(fasta_file, "fasta"))
    seq_count = len(sequences)
    seq_length = len(str(sequences[0].seq))
    for seq in sequences:
        if len(str(seq.seq)) != seq_length:
            raise ValueError(f"All sequences must have the same length. "
                             f"Sequence {seq.id} has length {len(str(seq.seq))}, expected {seq_length}.")
    with open(phylip_file, 'w') as outfile:
        outfile.write(f"{seq_count} {seq_length}\n")
        for seq in sequences:
            new_name = seq.id[-8:].rjust(10)  
            outfile.write(f"{new_name}  {str(seq.seq)}\n")

    print(f"FASTA file '{fasta_file}' successfully converted to Phylip format as '{phylip_file}'.")
def convert_fas_phy_all(codon_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(target_folder):
            pass
        else:
            raise
    for filename in os.listdir(codon_folder):
        file_name=filename.rsplit(".",1)[0]
        file_path = os.path.join(codon_folder, filename)
        fasta_to_phylip(file_path,f"{target_folder}/{file_name}.phy")
convert_fas_phy_all("/mnt/maple/ymchen/gene_loss/preflow/codon_align","/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_new")

FASTA file '/mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001351.fa' successfully converted to Phylip format as '/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_new/OG0001351.phy'.
FASTA file '/mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001338.fa' successfully converted to Phylip format as '/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_new/OG0001338.phy'.
FASTA file '/mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001438.fa' successfully converted to Phylip format as '/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_new/OG0001438.phy'.
FASTA file '/mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001394.fa' successfully converted to Phylip format as '/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_new/OG0001394.phy'.
FASTA file '/mnt/maple/ymchen/gene_loss/preflow/codon_align/OG0001370.fa' successfully converted to Phylip format as '/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_new/OG0001370.phy'.
FASTA file '/mnt/maple/ymchen/gene_loss/preflow/co

In [5]:
### name revise 
def modify_phylip_names(input_file, output_file):
    with open(input_file, 'r') as infile:
        # 读取第一行，它包含序列数和序列长度
        header = infile.readline().strip()

        # 读取剩余的序列行
        sequences = infile.readlines()

    # 打开输出文件，准备写入修改后的内容
    with open(output_file, 'w') as outfile:
        # 首先写入头部信息
        outfile.write(header + '\n')

        # 遍历每条序列
        for seq in sequences:
            # 分割序列名称和序列部分
            parts = seq.strip().split(maxsplit=1)
            if len(parts) != 2:
                print(f"Skipping invalid line: {seq}")
                continue

            name, sequence = parts
            
            # 获取名称的后8个字符，保留序列部分
            new_name = name[-8:]
            
            # 确保名称和序列之间有至少2个空格，这对 Phylip 格式是强制要求的
            outfile.write(f"{new_name:<10}  {sequence}\n")

    print(f"Modified file saved as {output_file}")

def get_revise_phy(raw_folder,new_folder):
    try:
        os.makedirs(new_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(new_folder):
            pass
        else:
            raise
    for filename in os.listdir(raw_folder):
        file_name=filename.rsplit(".",1)[0]
        file_path = os.path.join(raw_folder, filename)
        modify_phylip_names(file_path, f"{new_folder}/{file_name}.phy")
get_revise_phy("/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy","/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_revise")

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line: 

Skipping invalid line:            ---------

Skipping invalid line:            ---------

Skipping invalid line:            GTGGGACGA

Skipping invalid line:            GTGGGACGA

Skipping invalid line:            GCGGGGCAT

Skipping invalid line:            GGACACGGA

Skipping invalid line:            GTGATCCGG

Skipping invalid line:            GCGGGCCAT

Skipping invalid line:            ---------

Skipping invalid line:            AGGGGTGTG

Skipping invalid line:            TCGGGACAT

Skipping invalid line:            GCAGGACAT

Skipping i

In [9]:
###run PAML to get dN/dS
def prepare_ctl_run(codon_align_phy,ctl_file,target_file):
    content = [f"seqfile = {codon_align_phy}  * sequence data file name", f"outfile = {target_file}       * main result file", "verbose = 0  * 1: detailed output (list sequences), 0: concise output ", "icode = 0  * 0:universal code; 1:mammalian mt; 2-10:see below ","weighting = 0  * weighting pathways between codons (0/1)? ","commonf3x4 = 0  * use one set of codon freqs for all pairs (0/1)?"]
    with open(ctl_file, 'w') as file:
        for line in content:
            file.write(line + '\n\n')
    os.system(f"/mnt/maple/ymchen/paml/bin/yn00 {ctl_file} > /dev/null 2>&1")
    
def PAML_workflow(phy_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(target_folder):
            pass
        else:
            raise
    os.system(f"cd {target_folder}")
    for filename in os.listdir(phy_folder):
        file_name=filename.rsplit(".",1)[0]
        file_path = os.path.join(phy_folder, filename)
        prepare_ctl_run(file_path,f"{target_folder}/yn00.ctl",f"{target_folder}/{file_name}.out")
PAML_workflow("/mnt/maple/ymchen/gene_loss/preflow/codon_align_phy_new","/mnt/maple/ymchen/gene_loss/preflow/yn00_result")    

In [4]:
###calculate result from PAML output
import re
import pandas as pd

def extract_dnds(yn_out, output_file):
    with open(yn_out, 'r') as f:
        lines = f.readlines()
    dnds_data = []
    name_pairs = []
    dnds_pattern = re.compile(r'^\s+\d+\s+\d+\s+(\S+)\s+(\S+)\s+\S+\s+\S+\s+\S+\s+(\S+)\s+\+\-\s+\S+\s+(\S+)\s+\+\-')
    name_pattern = re.compile(r'(\S+)\s+vs\.\s+(\S+)')
    for line in lines:
        dnds_match = dnds_pattern.match(line)
        if dnds_match:
            gene1, gene2, dN, dS = dnds_match.groups()
            dnds_data.append([gene1, gene2, dN, dS])
        name_match = name_pattern.search(line)
        if name_match:
            gene1_name, gene2_name = name_match.groups()
            name_pairs.append([gene1_name, gene2_name])
    if dnds_data and name_pairs:
        result_data = []
        for i in range(len(dnds_data)):
            result_data.append(name_pairs[i] + dnds_data[i][2:])  # 合并名称对和dN/dS值
        df = pd.DataFrame(result_data, columns=["gene1", "gene2", "dN", "dS"])
        df.to_csv(output_file, sep='\t', index=False)
        print(f"dN/dS 结果已保存到 {output_file}")
    else:
        print("未找到任何 dN/dS 结果或基因名称对。")
def dNdS_workflow(yn00_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(target_folder):
            pass
        else:
            raise
    for filename in os.listdir(yn00_folder):
        file_name=filename.rsplit(".",1)[0]
        file_path = os.path.join(yn00_folder, filename)
        extract_dnds(file_path, f"{target_folder}/{file_name}")
dNdS_workflow("/mnt/maple/ymchen/gene_loss/preflow/yn00_result", "/mnt/maple/ymchen/gene_loss/preflow/dNdS_result")

dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0000899
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001383
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001149
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0000916
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001140
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001430
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001391
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001121
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001186
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001227
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001339
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001000
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0000874
dN/dS 结果已保存到 /mnt/maple/ymchen/gene_loss/preflow/dNdS_result/OG0001306
dN/dS 